In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
%%capture
class_names = ['cat', 'dog']
split = ['train[:70%]', 'train[70%:90%]', 'train[90%:]']

train_dataset, test_dataset, val_dataset = tfds.load(name='cats_vs_dogs', split=split, as_supervised=True)

In [ ]:
train_dataset = train_dataset.map(lambda x, y: (tf.image.resize(x, [180, 180]), y)).batch(32)
test_dataset = test_dataset.map(lambda x, y: (tf.image.resize(x, [180, 180]), y)).batch(32)
val_dataset = val_dataset.map(lambda x, y: (tf.image.resize(x, [180, 180]), y)).batch(32)

In [ ]:
inputs = tf.keras.layers.Input(shape=(180, 180, 3))
norm_input = tf.keras.layers.Rescaling(1./255)(inputs)

fmaps_1 = tf.keras.layers.Conv2D(
    filters=32, kernel_size=3, activation="relu")(norm_input)
pfmaps_1 = tf.keras.layers.MaxPool2D(pool_size=2)(fmaps_1)

fmaps_2 = tf.keras.layers.Conv2D(
    filters=64, kernel_size=3, activation="relu")(pfmaps_1)
pfmaps_2 = tf.keras.layers.MaxPool2D(pool_size=2)(fmaps_2)

fmaps_3 = tf.keras.layers.Conv2D(
    filters=128, kernel_size=3, activation="relu")(pfmaps_2)
pfmaps_3 = tf.keras.layers.MaxPool2D(pool_size=2)(fmaps_3)

fmaps_4 = tf.keras.layers.Conv2D(
    filters=256, kernel_size=3, activation="relu")(pfmaps_3)
pfmaps_4 = tf.keras.layers.MaxPool2D(pool_size=2)(fmaps_4)

fmaps_5 = tf.keras.layers.Conv2D(
    filters=256, kernel_size=3, activation="relu")(pfmaps_4)
pfmaps_5 = tf.keras.layers.MaxPool2D(pool_size=2)(fmaps_5)

flattened = tf.keras.layers.Flatten()(pfmaps_5)
output = tf.keras.layers.Dense(1, activation="sigmoid")(flattened)

In [ ]:
model = tf.keras.Model(
    inputs=inputs,
    outputs=output
)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss="binary_crossentropy",
    optimizer="rmsprop",
    metrics=["accuracy"]
)

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath="convnet",
        save_best_only=True,
        monitor="val_loss"
    )
]

In [ ]:
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=val_dataset,
    callbacks=callbacks
)

In [ ]:
m = tf.keras.models.load_model("convnet")
m.evaluate(test_dataset)